# Annotation Platform

## 1. Create Pipeline

In [ ]:
import dtlpy as dl

project_id = 'project_id'
pipeline_name = 'my pipeline'

project = dl.projects.get(project_id=project_id)
pipeline = project.pipelines.create(pipeline_name=pipeline_name)

Refresh the page and open the pipeline in edit mode

## 2. Add Dataset Node to the Pipeline

First create a dataset using the sdk

In [ ]:
import dtlpy as dl

project_id = 'project_id'
dataset_name = 'my dataset'

project = dl.projects.get(project_id=project_id)
dataset = project.datasets.create(dataset_name=dataset_name)

Add the dataset as a dataset node in the pipeline

In [ ]:
import dtlpy as dl

project_id = 'project_id'
dataset_name = 'my dataset'
pipeline_name = 'my pipeline'

project = dl.projects.get(project_id=project_id)
dataset = project.datasets.get(dataset_name=dataset_name)
pipeline = project.pipelines.get(pipeline_name=pipeline_name)

dataset_node = dl.DatasetNode(name=dataset_name,
                              project_id=project.id,
                              dataset_id=dataset.id,
                              position=(1, 1))
pipeline.nodes.add(node=dataset_node)
pipeline.update()

Refresh the page

## 3. Add Annotation Task Node to the Pipeline

In [ ]:
import dtlpy as dl

project_id = 'project_id'
dataset_name = 'my dataset'
pipeline_name = 'my pipeline'

task_name = 'annotation task'
my_user = dl.info()["user_email"]
task_owner = my_user
assignee_ids = [my_user]

project = dl.projects.get(project_id=project_id)
dataset = project.datasets.get(dataset_name=dataset_name)
pipeline = project.pipelines.get(pipeline_name=pipeline_name)

recipe = dataset.recipes.list()[0]
workload = dl.Workload.generate(assignee_ids=assignee_ids).workload

task_node = dl.TaskNode(name=task_name,
                        project_id=project.id,
                        dataset_id=dataset.id,
                        recipe_title=recipe.title,
                        recipe_id=recipe.id,
                        task_owner=task_owner,
                        workload=workload,
                        task_type='annotation',
                        position=(2, 1))
pipeline.nodes.add(node=task_node)
pipeline.update()

Refresh the page

## 4. Add QA Task Node to the Pipeline

In [ ]:
import dtlpy as dl

project_id = 'project_id'
dataset_name = 'my dataset'
pipeline_name = 'my pipeline'

task_name = 'qa task'
my_user = dl.info()["user_email"]
task_owner = my_user
assignee_ids = [my_user]

project = dl.projects.get(project_id=project_id)
dataset = project.datasets.get(dataset_name=dataset_name)
pipeline = project.pipelines.get(pipeline_name=pipeline_name)

recipe = dataset.recipes.list()[0]
workload = dl.Workload.generate(assignee_ids=assignee_ids).workload

task_node = dl.TaskNode(name=task_name,
                        project_id=project.id,
                        dataset_id=dataset.id,
                        recipe_title=recipe.title,
                        recipe_id=recipe.id,
                        task_owner=task_owner,
                        workload=workload,
                        task_type='qa',
                        position=(3, 1))
pipeline.nodes.add(node=task_node)
pipeline.update()

Refresh the page

## 5. Connect the Pipeline Nodes

In [ ]:
import dtlpy as dl

project_id = 'project_id'
pipeline_name = 'my pipeline'

dataset_node_name = 'my dataset'
annotation_task_node_name = 'annotation task'
qa_task_node_name = 'qa task'

project = dl.projects.get(project_id=project_id)
pipeline = project.pipelines.get(pipeline_name=pipeline_name)

dataset_node = None
annotation_task_node = None
qa_task_node = None
for node in pipeline.nodes:
    if node.name == dataset_node_name:
        dataset_node = node
    elif node.name == annotation_task_node_name:
        annotation_task_node = node
    elif node.name == qa_task_node_name:
        qa_task_node = node
    else:
        continue

dataset_node.connect(
    node=annotation_task_node,
    source_port=dataset_node.outputs[0],
    target_port=annotation_task_node.inputs[0],
)
annotation_task_node.connect(
    node=qa_task_node,
    source_port=annotation_task_node.outputs[0],
    target_port=qa_task_node.inputs[0],
    action='completed'
)
pipeline.update()

Refresh the page

## 6. Add Trigger to the Dataset Node

In [ ]:
import dtlpy as dl

project_id = 'project_id'
pipeline_name = 'my pipeline'

dataset_node_name = 'my dataset'

project = dl.projects.get(project_id=project_id)
pipeline = project.pipelines.get(pipeline_name=pipeline_name)

dataset_node = None
for node in pipeline.nodes:
    if node.name == dataset_node_name:
        dataset_node = node
    else:
        continue

dataset_node.add_trigger(
    trigger_type=dl.TriggerType.EVENT,
    resource=dl.TriggerResource.ITEM,
    actions=dl.TriggerAction.CREATED,
    execution_mode=dl.TriggerExecutionMode.ONCE
)
pipeline.update()

Refresh the page

## 7. Install the Pipeline

In [ ]:
import dtlpy as dl

project_id = 'project_id'
pipeline_name = 'my pipeline'

project = dl.projects.get(project_id=project_id)
pipeline = project.pipelines.get(pipeline_name=pipeline_name)

pipeline.install()

Refresh the page and change to View Mode

## 9. Trigger the pipeline by uploading an Item